In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/COVID-19_Radiography_Dataset"

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
import glob
import matplotlib.pyplot as plt
import cv2

In [ ]:
train = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Train" 
val ="/content/drive/MyDrive/COVID-19_Radiography_Dataset/Val"
test = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Test"

gen_train = ImageDataGenerator(rescale = 1./255., zoom_range = 0.2,)
gen_val = ImageDataGenerator(rescale = 1./255.)
gen_test = ImageDataGenerator(rescale = 1./255.)

train_gen = gen_train.flow_from_directory(train, batch_size = 64, class_mode = 'categorical', target_size = (220, 220))
validation_gen = gen_val.flow_from_directory(val, batch_size = 64, class_mode = 'categorical', target_size=(220, 220))
test_gen = gen_test.flow_from_directory(test, shuffle = False, batch_size=64, class_mode = 'categorical', target_size=(220, 220))

Found 5786 images belonging to 2 classes.
Found 722 images belonging to 2 classes.
Found 722 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.ResNet50V2(weights = 'imagenet', input_shape = (220, 220,3), include_top = False)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))
model.summary()

94668760/94668760 [==============================] - 5s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 24,089,858
Trainable params: 525,058
Non-tr

In [ ]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2,restore_best_weights=True)

history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 40, epochs = 11, callbacks = callback)

Epoch 1/11
40/40 [==============================] - 1530s 38s/step - loss: 0.4039 - accuracy: 0.8279 - val_loss: 0.3227 - val_accuracy: 0.8338
Epoch 2/11
40/40 [==============================] - 575s 14s/step - loss: 0.2868 - accuracy: 0.8750 - val_loss: 0.3675 - val_accuracy: 0.8158
Epoch 3/11
40/40 [==============================] - 325s 8s/step - loss: 0.2655 - accuracy: 0.8918 - val_loss: 0.2974 - val_accuracy: 0.8435
Epoch 4/11
40/40 [==============================] - 195s 5s/step - loss: 0.2255 - accuracy: 0.9047 - val_loss: 0.2455 - val_accuracy: 0.8809
Epoch 5/11
40/40 [==============================] - 116s 3s/step - loss: 0.2414 - accuracy: 0.9016 - val_loss: 0.3016 - val_accuracy: 0.8366
Epoch 6/11
40/40 [==============================] - 83s 2s/step - loss: 0.2261 - accuracy: 0.9060 - val_loss: 0.3197 - val_accuracy: 0.8463
Epoch 7/11
40/40 [==============================] - 73s 2s/step - loss: 0.2115 - accuracy: 0.9151 - val_loss: 0.2351 - val_accuracy: 0.8864
Epoch 8/11
4

In [ ]:
history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 40, epochs = 1, callbacks = callback)

40/40 [==============================] - 44s 1s/step - loss: 0.2124 - accuracy: 0.9120 - val_loss: 0.3362 - val_accuracy: 0.8449


In [ ]:
np.save("ResNet50V2-history.npy",history.history)

In [ ]:
model.save("ResNet50V2.h5")

In [ ]:
y_pred = model.predict(test_gen)

In [ ]:
y_pred_list = []

for i in range(len(y_pred)):
    y_pred[i] = np.where(y_pred[i] == np.amax(y_pred[i]))
    y_pred_list.append(y_pred[i][0])

y_pred_list

<ipython-input-11-b51e2648bcaf>:4: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  y_pred[i] = np.where(y_pred[i] == np.amax(y_pred[i]))


[1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0

In [ ]:
from google.colab import files 
files.download("ResNet50V2.h5")

files.download("ResNet50V2-history.npy.h5")

In [ ]:
files.download("ResNet50V2-history.npy")